In [4]:
import random
import spacy
from spacy.training import Example
from tqdm import tqdm
from spacy import displacy

ImportError: cannot import name 'ModelMetaclass' from 'pydantic.main' (C:\Users\HP\AppData\Roaming\Python\Python39\site-packages\pydantic\main.py)

In [17]:
# Define the training data directly in the code
TRAIN_DATA = [
("The ripe, red strawberries glistened in the morning.",{"entities":[(10,13,"COLOR")]}),
(" sun.The sky turned a deep shade of blue as the day transitioned into evening.",{"entities":[(36,40,"COLOR")]}),
("The clear blue waters of the ocean stretched as far as the eye could see.",{"entities":[(10,14,"COLOR")]}),
("The clear blue sky contrasted beautifully with the vibrant yellow sunflowers in the field.",{"entities":[(10,14,"COLOR"),(59,65,"COLOR")]}),
("The taxi driver noticed the yellow traffic light ahead and began to slow down, preparing to stop.",{"entities":[(28,34,"COLOR")]})
]

labels = ["COLOR"]

nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

for ent in labels:
    ner.add_label(ent)

In [18]:
epochs = 1000
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()

    for i in tqdm(range(epochs)):
        random.shuffle(TRAIN_DATA)
        for text, annotation in TRAIN_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotation)
            nlp.update([example], sgd=optimizer)

100%|██████████| 1000/1000 [05:48<00:00,  2.87it/s]


In [19]:
# Test with a simple example
doc = nlp("the sky is blue")
print(doc.ents)
for token in doc:
    print(f"Token: {token.text}, NER: {token.ent_type_}")
# Test with another simple example
doc2 = nlp("The ripe bananas were a vibrant shade of yellow.")
print(doc2.ents)
for token in doc2:
    print(f"Token: {token.text}, NER: {token.ent_type_}")
displacy.render(doc, style="ent", jupyter=True)
displacy.render(doc2, style="ent", jupyter=True)

(blue,)
Token: the, NER: 
Token: sky, NER: 
Token: is, NER: 
Token: blue, NER: COLOR
(yellow,)
Token: The, NER: 
Token: ripe, NER: 
Token: bananas, NER: 
Token: were, NER: 
Token: a, NER: 
Token: vibrant, NER: 
Token: shade, NER: 
Token: of, NER: 
Token: yellow, NER: COLOR
Token: ., NER: 


In [20]:


nlp.to_disk('color_nlp')

In [21]:
nlp_color = spacy.load("color_nlp")



In [22]:
doc2 = nlp_color("The ripe bananas were a vibrant shade of yellow.")
print(doc2.ents)
for token in doc2:
    print(f"Token: {token.text}, NER: {token.ent_type_}")
displacy.render(doc2, style="ent", jupyter=True)

(yellow,)
Token: The, NER: 
Token: ripe, NER: 
Token: bananas, NER: 
Token: were, NER: 
Token: a, NER: 
Token: vibrant, NER: 
Token: shade, NER: 
Token: of, NER: 
Token: yellow, NER: COLOR
Token: ., NER: 
